# Setting up Python and Pathway

Pathway can be installed to a Python 3.10 environment using pip, please register at https://pathway.com to get beta access to the package

In [ ]:
PIP_PACKAGE_ADDRESS=""
if not PIP_PACKAGE_ADDRESS:
    print(
        "Please register at https://pathway.com/developers/documentation/introduction/installation-and-first-steps\n"
        "To get the pip package installation link!"
    )

In [ ]:
if not (sys.version_info.major==3 and sys.version_info.minor==10):
    raise Exception("Pathway is only built for Python 3.10 at the moment")

In [ ]:
# Install pathway's package
!pip install {PIP_PACKAGE_ADDRESS} 1>/dev/null 2>/dev/null

# Playing with joins.
A brief explanation on how to perform joins with Pathway.

Join is one of the basic table operation provided in Pathway.
A join operation combines columns from two different tables by associating rows with matching some given values both tables.

This guide presents several samples of code using the joins:
*  [join (inner join)](#simple-join)
*  [left_join, right_join (outer_joins)](#outer-joins)

The examples demonstrate usual use-cases, explain the behavior of
outer joins, and point out some peculiarities you may encounter
while using Pathway.

Soon, this guide will also show examples of code that:
*  [joins using foreign keys](#joins-on-foreign-key)
*  [uses joins in a chain](#chaining-joins)

## Prerequisites

Be sure to import Pathway.

In [1]:
import pathway as pw


Also, we need some tables to play with. For the sake of presentation,
let us consider a following made up scenario: a group of four friends
(table friends) goes to grab some breakfast.


In [2]:
friends = pw.debug.table_from_markdown(
    """
    | name  |budget
  1 | Alice | 13
  2 | Bob   | 10
  3 | Carol | 11
  4 | Dan   | 12
 """
)
menu = pw.debug.table_from_markdown(
    """
    | dish                              | price
  1 | pancakes_with_jam                 | 11
  2 | pb_jam_sandwich                   | 9
  3 | jam_pb_pancakes                   | 12
  4 | scrambled_egg                     | 11
 """
)

#
However, some of them have allergies (table allergies),
and cannot eat everything. Luckily, the restaurant has a list
of allergens contained in each dish (table allergens_in_menu).


In [3]:
allergies = pw.debug.table_from_markdown(
    """
    | person| allergy
  1 | Alice | milk
  2 | Bob   | eggs
  3 | Carol | peanuts
  4 | Carol | milk
 """
)

allergens_in_menu = pw.debug.table_from_markdown(
    """
    | dish                              | allergen
  1 | pancakes_with_jam                 | milk
  2 | pancakes_with_jam                 | eggs
  3 | pb_jam_sandwich                   | peanuts
  4 | jam_pb_pancakes                   | eggs
  5 | jam_pb_pancakes                   | peanuts
  6 | jam_pb_pancakes                   | milk
  7 | scrambled_egg                     | eggs
 """
)

## Simple join.
### Syntax
Informally, the syntax of join is:

In [4]:
# _MD_SHOW_table.join(other, *on)

where
* `table` and `other` are tables to be joined,
* `*on` is a list of conditions of form:

In [5]:
# _MD_SHOW_table.some_column == other.some_other_column

**Remark:** the order of tables in the condition matters. That is,
a condition of the following form won't be accepted:

In [6]:
# _MD_SHOW_other.some_other_column == table.some_column

### Examples
As a warm-up, let us see how to make a simple join-queries.
Let's begin with simple query that find all pairs person-dish
that can put someone in a hospital.

In [7]:
bad_match = allergies.join(
    allergens_in_menu, allergies.allergy == allergens_in_menu.allergen
).select(allergens_in_menu.dish, allergies.person)
pw.debug.compute_and_print(bad_match)

            | dish              | person
^F3S17BY... | jam_pb_pancakes   | Alice
^3MZEF3A... | jam_pb_pancakes   | Bob
^1XAC7SA... | jam_pb_pancakes   | Carol
^ATT8C1A... | jam_pb_pancakes   | Carol
^VJ3K9DF... | pancakes_with_jam | Alice
^V1RPZW8... | pancakes_with_jam | Bob
^NZTRM84... | pancakes_with_jam | Carol
^R0GE4WM... | pb_jam_sandwich   | Carol
^X8BJ183... | scrambled_egg     | Bob


#
As we can see, the result is a table of life-threatening pairs.

The `select` function works here similarly as `select` on a table. The difference is
that here, we can use columns of both tables as arguments, e.g.

In [8]:
def make_food_constraint_note(name, dish):
    return f"{name} can't eat {dish}."


bad_match_note = allergies.join(
    allergens_in_menu, allergies.allergy == allergens_in_menu.allergen
).select(
    note=pw.apply(make_food_constraint_note, allergies.person, allergens_in_menu.dish)
)
pw.debug.compute_and_print(bad_match_note)

            | note
^F3S17BY... | Alice can't eat jam_pb_pancakes.
^VJ3K9DF... | Alice can't eat pancakes_with_jam.
^3MZEF3A... | Bob can't eat jam_pb_pancakes.
^V1RPZW8... | Bob can't eat pancakes_with_jam.
^X8BJ183... | Bob can't eat scrambled_egg.
^1XAC7SA... | Carol can't eat jam_pb_pancakes.
^ATT8C1A... | Carol can't eat jam_pb_pancakes.
^NZTRM84... | Carol can't eat pancakes_with_jam.
^R0GE4WM... | Carol can't eat pb_jam_sandwich.


*Remark:* note that id is now some auto generated number, which is the usual behavior of join.

## Outer joins
The difference between outer joins and joins is that the outer join adds to the result
table also entries that didn't match:

In [9]:
# _MD_SHOW_left.left_join(right, conditions)

includes entries from the *left* table and

In [10]:
# _MD_SHOW_left.right_join(right, conditions)

includes entries from the *right* table.

### Simple example
To demonstrate outer-joins, we can consider a question about forbidden
breakfast configurations (i.e. pairs person-dish, such that person is
allergic to some ingredients).

In [11]:
people_allergies = friends.left_join(
    allergies, friends.name == allergies.person
).select(friends.name, allergies.allergy)
pw.debug.compute_and_print(people_allergies)

forbidden_breakfast = people_allergies.left_join(
    allergens_in_menu, people_allergies.allergy == allergens_in_menu.allergen
).select(people_allergies.name, allergens_in_menu.dish)
pw.debug.compute_and_print(forbidden_breakfast)

            | name  | allergy
^VJ3K9DF... | Alice | milk
^V1RPZW8... | Bob   | eggs
^M8FN8EJ... | Carol | milk
^R0GE4WM... | Carol | peanuts
^MZ49F0F... | Dan   | 
            | name  | dish
^XPE17AC... | Alice | jam_pb_pancakes
^23V2D62... | Alice | pancakes_with_jam
^44G774K... | Bob   | jam_pb_pancakes
^8REXNF0... | Bob   | pancakes_with_jam
^5ZXSYKF... | Bob   | scrambled_egg
^99X8JSV... | Carol | jam_pb_pancakes
^N8KHY1W... | Carol | jam_pb_pancakes
^T4MA9R8... | Carol | pancakes_with_jam
^TXDF4CV... | Carol | pb_jam_sandwich
^2YAV0VF... | Dan   | 


Now, as we can see, the table has duplicate rows. This is intended behavior, as
a particular person can be allergic to more than one ingredient. In other words,
when we add a reason column to our result table, we can see that each row was
included in the table above for a different reason.

In [12]:
forbidden_breakfast = people_allergies.left_join(
    allergens_in_menu, people_allergies.allergy == allergens_in_menu.allergen
).select(
    people_allergies.name, allergens_in_menu.dish, reason=allergens_in_menu.allergen
)
pw.debug.compute_and_print(forbidden_breakfast)

            | name  | dish              | reason
^XPE17AC... | Alice | jam_pb_pancakes   | milk
^23V2D62... | Alice | pancakes_with_jam | milk
^44G774K... | Bob   | jam_pb_pancakes   | eggs
^8REXNF0... | Bob   | pancakes_with_jam | eggs
^5ZXSYKF... | Bob   | scrambled_egg     | eggs
^N8KHY1W... | Carol | jam_pb_pancakes   | milk
^99X8JSV... | Carol | jam_pb_pancakes   | peanuts
^T4MA9R8... | Carol | pancakes_with_jam | milk
^TXDF4CV... | Carol | pb_jam_sandwich   | peanuts
^2YAV0VF... | Dan   |                   | 


### Removing duplicates
If we really want to have a table without the 'reason' column and without
duplicates, we can achieve that with extra `groupby` and `reduce` :

In [13]:
forbidden_breakfast = people_allergies.left_join(
    allergens_in_menu, people_allergies.allergy == allergens_in_menu.allergen
).select(people_allergies.name, allergens_in_menu.dish)

forbidden_breakfast_no_duplicates = forbidden_breakfast.groupby(
    forbidden_breakfast.name, forbidden_breakfast.dish
).reduce(forbidden_breakfast.name, forbidden_breakfast.dish)

pw.debug.compute_and_print(forbidden_breakfast_no_duplicates)

            | name  | dish
^M8E4DFK... | Alice | jam_pb_pancakes
^JMHBFKG... | Alice | pancakes_with_jam
^AN7YRTF... | Bob   | jam_pb_pancakes
^GZ0TEZ6... | Bob   | pancakes_with_jam
^W8AZSTG... | Bob   | scrambled_egg
^DBZV8YR... | Carol | jam_pb_pancakes
^N585E7A... | Carol | pancakes_with_jam
^9DJG3GD... | Carol | pb_jam_sandwich
^30JAJCE... | Dan   | 


That simple application of groupby-reduce combination essentially selects
unique set of pairs (name, dish) from our table. More on the reduce-groupby operations
can be found in the reduce-groupby [manual](/developers/documentation/table-operations/groupby-reduce-manual).

### Computed and omitted columns.
Because left join operation returns rows from the left table, even if there is no
match in the right column, some columns can't be computed.

**The left join computes values in all columns that can be
computed, and leaves all the remaining columns empty.**

As an example, we can consider a modified variant of the `bad_match_note` table.
The modification is that we want to include all the
people, and additionally we want to display their budget.

To that end, it is enough that we join the `friends` table with the `bad_match` table, as
together they include all necessary information.

In [14]:
basic_customer_info = friends.left_join(
    bad_match, friends.name == bad_match.person
).select(
    friends.name,
    friends.budget,
    note=pw.apply(make_food_constraint_note, friends.name, bad_match.dish),
)
pw.debug.compute_and_print(basic_customer_info)

            | name  | budget | note
^MK7AE5W... | Alice | 13     | Alice can't eat jam_pb_pancakes.
^R90GTQS... | Alice | 13     | Alice can't eat pancakes_with_jam.
^0WRP6GY... | Bob   | 10     | Bob can't eat jam_pb_pancakes.
^ZS7XXFT... | Bob   | 10     | Bob can't eat pancakes_with_jam.
^6WXMMZB... | Bob   | 10     | Bob can't eat scrambled_egg.
^DYBRNS6... | Carol | 11     | Carol can't eat jam_pb_pancakes.
^PJGD2SR... | Carol | 11     | Carol can't eat jam_pb_pancakes.
^HKQXPXX... | Carol | 11     | Carol can't eat pancakes_with_jam.
^5KSXXGY... | Carol | 11     | Carol can't eat pb_jam_sandwich.
^MZ49F0F... | Dan   | 12     | 


To once again demonstrate the behavior, let us change `friends.name` to
`bad_match.person` in the select part of our code.

In [15]:
basic_customer_info = friends.left_join(
    bad_match, friends.name == bad_match.person
).select(
    bad_match.person,
    friends.budget,
    note=pw.apply(make_food_constraint_note, friends.name, bad_match.dish),
)
pw.debug.compute_and_print(basic_customer_info)

            | person | budget | note
^MZ49F0F... |        | 12     | 
^MK7AE5W... | Alice  | 13     | Alice can't eat jam_pb_pancakes.
^R90GTQS... | Alice  | 13     | Alice can't eat pancakes_with_jam.
^0WRP6GY... | Bob    | 10     | Bob can't eat jam_pb_pancakes.
^ZS7XXFT... | Bob    | 10     | Bob can't eat pancakes_with_jam.
^6WXMMZB... | Bob    | 10     | Bob can't eat scrambled_egg.
^DYBRNS6... | Carol  | 11     | Carol can't eat jam_pb_pancakes.
^PJGD2SR... | Carol  | 11     | Carol can't eat jam_pb_pancakes.
^HKQXPXX... | Carol  | 11     | Carol can't eat pancakes_with_jam.
^5KSXXGY... | Carol  | 11     | Carol can't eat pb_jam_sandwich.


Now, the cell that earlier included Dan is empty, as value of this column is defined
by column in `bad_match` table. Since there is no match, person was not filled in.
Budget is still computable though, hence it is still present.

### On right joins
In the examples above, we only relied on `left_join`. The `right_join` operation is quite
similar in its behavior. Namely,

In [16]:
people_allergies = friends.left_join(
    allergies, friends.name == allergies.person
).select(friends.name, allergies.allergy)
pw.debug.compute_and_print(people_allergies)

            | name  | allergy
^VJ3K9DF... | Alice | milk
^V1RPZW8... | Bob   | eggs
^M8FN8EJ... | Carol | milk
^R0GE4WM... | Carol | peanuts
^MZ49F0F... | Dan   | 


is equivalent to

In [17]:
people_allergies = allergies.right_join(
    friends, allergies.person == friends.name
).select(friends.name, allergies.allergy)
pw.debug.compute_and_print(people_allergies)

            | name  | allergy
^VJ3K9DF... | Alice | milk
^V1RPZW8... | Bob   | eggs
^ABST817... | Carol | milk
^R0GE4WM... | Carol | peanuts
^KARHAYN... | Dan   | 


When we join two tables, the only difference is in syntax - since allergies is
the table on which we call `join_right`, it must be first argument
in the join condition, i.e. it is

In [18]:
# _MD_SHOW_allergies.person == friends.name

as opposed to

In [19]:
# _MD_SHOW_friends.name == allergies.person

used in the `left_join`.

## Full outer-joins:
coming soon.

## Joins on foreign key:
coming soon

## Chaining joins:
coming soon.